# Emotion Classification Model - IPYNB

pip install numpy tensorflow scikit-learn matplotlib

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import layers, models, regularizers
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
# Define parameters
IMG_SIZE = (48, 48)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
l2_strength = 0.001  # L2 regularization strength

- Modify this based on the path of the training & testing dataset

In [3]:
# Directories
train_dir = 'C:/Users/Melvin Tang/OneDrive/Codes/I2ML/FINALPRO/archive/train'

test_dirs = [
    'C:/Users/Melvin Tang/OneDrive/Codes/I2ML/FINALPRO/archive/testgeli',
    'C:/Users/Melvin Tang/OneDrive/Codes/I2ML/FINALPRO/archive/testkevin',
    'C:/Users/Melvin Tang/OneDrive/Codes/I2ML/FINALPRO/archive/test'
]


In [5]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=17,
    horizontal_flip=True, 
    validation_split=VALIDATION_SPLIT,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode='rgb',  # Keeping RGB as in your original code
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 20011 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.


In [6]:
# Categories for the classes
categories = list(train_generator.class_indices.keys())

# Compute Class Weights
labels = train_generator.classes
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

In [7]:
# Model Definition
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength), input_shape=(48, 48, 3)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  # Reduced dropout rate

    # Second Convolutional Block
    layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),  # Reduced dropout rate

    # Third Convolutional Block
    layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),  # Reduced dropout rate

    # Fourth Convolutional Block (Added)
    layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),  # Maintained dropout rate

    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(512, kernel_initializer='he_normal',
                 kernel_regularizer=regularizers.l2(l2_strength)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),  # Reduced dropout rate
    layers.Dense(5, activation='softmax')
])

- Model Compiling, Loss Functions and Callbacks 
(Utilized early stopping and learning rate reducer)

In [8]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)  # Reduced learning rate

# Using label smoothing
loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=75,
    validation_data=validation_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/75
626/626 [==============================] - 209s 321ms/step - loss: 6.1923 - accuracy: 0.2250 - val_loss: 5.0441 - val_accuracy: 0.3178 - lr: 5.0000e-04
Epoch 2/75
626/626 [==============================] - 19s 30ms/step - loss: 4.2858 - accuracy: 0.3159 - val_loss: 3.3090 - val_accuracy: 0.4196 - lr: 5.0000e-04
Epoch 3/75
626/626 [==============================] - 19s 30ms/step - loss: 2.8745 - accuracy: 0.4227 - val_loss: 2.4118 - val_accuracy: 0.3912 - lr: 5.0000e-04
Epoch 4/75
626/626 [==============================] - 21s 33ms/step - loss: 2.1762 - accuracy: 0.4696 - val_loss: 1.9286 - val_accuracy: 0.4972 - lr: 5.0000e-04
Epoch 5/75
626/626 [==============================] - 23s 37ms/step - loss: 1.8634 - accuracy: 0.5041 - val_loss: 1.7763 - val_accuracy: 0.4812 - lr: 5.0000e-04
Epoch 6/75
626/626 [==============================] - 21s 33ms/step - loss: 1.7477 - accuracy: 0.5200 - val_loss: 1.6885 - val_accuracy: 0.5088 - lr: 5.0000e-04
Epoch 7/75
626/626 [============

In [ ]:
# Plotting functions
def plot_accuracy(history):
    plt.figure(figsize=(8,6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_loss(history):
    plt.figure(figsize=(8,6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_accuracy(history)
plot_loss(history)

# Retrieve the final training and validation accuracy and loss
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

# Print the final training and validation accuracy and loss
print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

In [ ]:
# Function to create a test data generator for a given directory
def create_test_generator(test_dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        color_mode='rgb',  # Keeping RGB as in your original code
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    return test_generator

# Evaluate the model on multiple test sets
def evaluate_test_set(model, test_generator, test_dir, categories):
    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Directory: {test_dir}')
    print(f'Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}')

    # Predictions and classification report
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    print(classification_report(true_classes, predicted_classes, target_names=categories))

    # Confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {os.path.basename(test_dir)}')
    plt.show()

# Loop through each test directory and evaluate
for test_dir in test_dirs:
    test_generator = create_test_generator(test_dir)
    evaluate_test_set(model, test_generator, test_dir, categories)

In [ ]:
# After training your model or loading it
model.save('C:/Users/LENOVO/Downloads/real-time-emotion-recognition/cnn_trial.h5')  # Save it as .h5 format